In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q trl xformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
from datasets import load_dataset

dataset = load_dataset("rohansolo/BB_HindiHinglishV2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train_sft split:   0%|          | 0/199137 [00:00<?, ? examples/s]

Generating test_sft split:   0%|          | 0/49785 [00:00<?, ? examples/s]

In [4]:
print(dataset['train_sft'])

Dataset({
    features: ['id', 'messages', 'category', '__index_level_0__'],
    num_rows: 199137
})


In [28]:
train_dataset = dataset['train_sft'][:8]

In [31]:
# train_dataset

In [5]:
import pandas as pd
df = pd.DataFrame(dataset['train_sft'][:8])
df.head()

,id,messages,category,__index_level_0__
0,28782,[{'content': 'आपको कुछ गणितीय समीकरण दिए गए है...,None,25706
1,46621,"[{'content': '""वह पांच दिनों से काम करता है"" क...",None,157556
2,25491,"[{'content': 'दो संख्याओं को दिए गए होने पर, ए...",None,125384
3,37734,[{'content': 'इस खतरनाक माल के लिए एक चेतावनी ...,None,5877
4,59232,[{'content': 'पेपर को रीसाइकल करने की प्रक्रिय...,None,17070


In [6]:
train = df['messages']
train = pd.DataFrame(train,columns =['messages'] )
train.rename(columns = {"messages":"text"},inplace=True)
train

,text
0,[{'content': 'आपको कुछ गणितीय समीकरण दिए गए है...
1,"[{'content': '""वह पांच दिनों से काम करता है"" क..."
2,"[{'content': 'दो संख्याओं को दिए गए होने पर, ए..."
3,[{'content': 'इस खतरनाक माल के लिए एक चेतावनी ...
4,[{'content': 'पेपर को रीसाइकल करने की प्रक्रिय...
5,[{'content': 'खुराक बदलने से जुड़े एक खतरा का ...
6,[{'content': 'एक छोड़ी हुई भूलभुलैया में जहां ...
7,[{'content': 'अफ्रीका में सबसे बड़ी झील का नाम...


In [7]:
# train['text'][0]

# 1. Bits and bytes configuration

In [8]:
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from transformers import BitsAndBytesConfig
import torch
from transformers import AutoModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer

bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,)

model = AutoModelForCausalLM.from_pretrained(
   'mistralai/Mistral-7B-Instruct-v0.2',
    quantization_config=bnb_config,
    device_map={"": 0})


model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

#2. Load Tokenizer

In [9]:


# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    padding_side="left",
     model_max_length=512,
    trust_remote_code=True)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

(True, True)

# 3. Add adapter to layer

In [10]:
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
        r=16,
        lora_alpha=16,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
    )
model = get_peft_model(model, peft_config)

In [ ]:
#Truncate the conversations as
tokenizer.apply_chat_template(train['text'][0],
                              return_tensors ="pt",
                              # truncation=True,
                              tokenize = False,

)

'<s>[INST] आपको कुछ गणितीय समीकरण दिए गए हैं जो किसी न किसी रूप से संबंधित लगते हैं। क्या आप उनमें छिपे रिश्ते को समझ सकते हैं और अधूरे समीकरण को पूरा कर सकते हैं?\n1. 2 + 3 = 10\n2. 7 + 8 = 46\n3. 6 + 4 = 30\n4. 1 + 9 = 28\n5. 9 + 2 = ? \nपरामर्श: नंबरों के बीच रिश्ता उनके महत्वों पर नहीं, बल्कि उनकी स्थिति पर आधारित है।\n [/INST]नंबरों का स्थान बदलने से उसका मूल्य बदल जाता है। ये समीकरण एक विशेष ढंग से तैयार किए गए हैं, जिसमें नंबरों को एक निश्चित प्रकार से आवंटित किया गया है और उनका मूल्य उस प्रकार के आवंटन से निर्धारित है।\n\nउदहारण के लिए, पहला समीकरण 2 + 3 = 10 है, जो कि सत्य नहीं है। इसमें छुपा रिश्ता यह है कि आप नंबर को अलग अलग टॉगल करने से एक समान प्रकार के आवंटन में मूल्य को बदल सकते हैं। इसी तरह, दूसरे समीकरण 7 + 8 = 46 में भी यही रिश्ता छुपा हुआ है। यह 6 से अधिक सात नमुनों में से एक है जो अलग-अलग निश्चित प्रकार के आवंटन में मूल्यों को अद्यतन करते हैं।\n\nआप दूसरे तीन समीकरणों के लिए भी इसी तरह के रिश्तों को ढूंढ सकते हैं और अधूरे समीकरण को पूरा कर सकते हैं। अंतिम समीकरण 9 +

In [ ]:
train['text'][0]

[{'content': 'आपको कुछ गणितीय समीकरण दिए गए हैं जो किसी न किसी रूप से संबंधित लगते हैं। क्या आप उनमें छिपे रिश्ते को समझ सकते हैं और अधूरे समीकरण को पूरा कर सकते हैं?\n1. 2 + 3 = 10\n2. 7 + 8 = 46\n3. 6 + 4 = 30\n4. 1 + 9 = 28\n5. 9 + 2 = ? \nपरामर्श: नंबरों के बीच रिश्ता उनके महत्वों पर नहीं, बल्कि उनकी स्थिति पर आधारित है।\n',
  'role': 'user'},
 {'content': 'नंबरों का स्थान बदलने से उसका मूल्य बदल जाता है। ये समीकरण एक विशेष ढंग से तैयार किए गए हैं, जिसमें नंबरों को एक निश्चित प्रकार से आवंटित किया गया है और उनका मूल्य उस प्रकार के आवंटन से निर्धारित है।\n\nउदहारण के लिए, पहला समीकरण 2 + 3 = 10 है, जो कि सत्य नहीं है। इसमें छुपा रिश्ता यह है कि आप नंबर को अलग अलग टॉगल करने से एक समान प्रकार के आवंटन में मूल्य को बदल सकते हैं। इसी तरह, दूसरे समीकरण 7 + 8 = 46 में भी यही रिश्ता छुपा हुआ है। यह 6 से अधिक सात नमुनों में से एक है जो अलग-अलग निश्चित प्रकार के आवंटन में मूल्यों को अद्यतन करते हैं।\n\nआप दूसरे तीन समीकरणों के लिए भी इसी तरह के रिश्तों को ढूंढ सकते हैं और अधूरे समीकरण को पूर

In [11]:
def create_prompt(text):
  encode = tokenizer.apply_chat_template(text,
                                         return_tensors = "pt",
                                         truncation = True,
                                         tokenize = False # use false to see the special tokens with text
                                         )
  return encode



In [12]:
create_prompt(train['text'][0])

'<s>[INST] आपको कुछ गणितीय समीकरण दिए गए हैं जो किसी न किसी रूप से संबंधित लगते हैं। क्या आप उनमें छिपे रिश्ते को समझ सकते हैं और अधूरे समीकरण को पूरा कर सकते हैं?\n1. 2 + 3 = 10\n2. 7 + 8 = 46\n3. 6 + 4 = 30\n4. 1 + 9 = 28\n5. 9 + 2 = ? \nपरामर्श: नंबरों के बीच रिश्ता उनके महत्वों पर नहीं, बल्कि उनकी स्थिति पर आधारित है।\n [/INST]नंबरों का स्थान बदलने से उसका मूल्य बदल जाता है। ये समीकरण एक विशेष ढंग से तैयार किए गए हैं, जिसमें नंबरों को एक निश्चित प्रकार से आवंटित किया गया है और उनका मूल्य उस प्रकार के आवंटन से निर्धारित है।\n\nउदहारण के लिए, पहला समीकरण 2 + 3 = 10 है, जो कि सत्य नहीं है। इसमें छुपा रिश्ता यह है कि आप नंबर को अलग अलग टॉगल करने से एक समान प्रकार के आवंटन में मूल्य को बदल सकते हैं। इसी तरह, दूसरे समीकरण 7 + 8 = 46 में भी यही रिश्ता छुपा हुआ है। यह 6 से अधिक सात नमुनों में से एक है जो अलग-अलग निश्चित प्रकार के आवंटन में मूल्यों को अद्यतन करते हैं।\n\nआप दूसरे तीन समीकरणों के लिए भी इसी तरह के रिश्तों को ढूंढ सकते हैं और अधूरे समीकरण को पूरा कर सकते हैं। अंतिम समीकरण 9 +

In [14]:
train['text']=train['text'].apply(create_prompt)

In [15]:
train['text'][0]

'<s>[INST] आपको कुछ गणितीय समीकरण दिए गए हैं जो किसी न किसी रूप से संबंधित लगते हैं। क्या आप उनमें छिपे रिश्ते को समझ सकते हैं और अधूरे समीकरण को पूरा कर सकते हैं?\n1. 2 + 3 = 10\n2. 7 + 8 = 46\n3. 6 + 4 = 30\n4. 1 + 9 = 28\n5. 9 + 2 = ? \nपरामर्श: नंबरों के बीच रिश्ता उनके महत्वों पर नहीं, बल्कि उनकी स्थिति पर आधारित है।\n [/INST]नंबरों का स्थान बदलने से उसका मूल्य बदल जाता है। ये समीकरण एक विशेष ढंग से तैयार किए गए हैं, जिसमें नंबरों को एक निश्चित प्रकार से आवंटित किया गया है और उनका मूल्य उस प्रकार के आवंटन से निर्धारित है।\n\nउदहारण के लिए, पहला समीकरण 2 + 3 = 10 है, जो कि सत्य नहीं है। इसमें छुपा रिश्ता यह है कि आप नंबर को अलग अलग टॉगल करने से एक समान प्रकार के आवंटन में मूल्य को बदल सकते हैं। इसी तरह, दूसरे समीकरण 7 + 8 = 46 में भी यही रिश्ता छुपा हुआ है। यह 6 से अधिक सात नमुनों में से एक है जो अलग-अलग निश्चित प्रकार के आवंटन में मूल्यों को अद्यतन करते हैं।\n\nआप दूसरे तीन समीकरणों के लिए भी इसी तरह के रिश्तों को ढूंढ सकते हैं और अधूरे समीकरण को पूरा कर सकते हैं। अंतिम समीकरण 9 +

In [16]:
# for i, j in train.iterrows():
#   # print(j)
#   print(f" row {i},\n\n {create_prompt(j[0])}")

In [ ]:
# text.clear()
# text

[]

In [37]:
# # train = train[:100]
# text = []
# for i, j in train.iterrows():
#   try:
#     # print(f" row {i},\n\n {create_prompt(j[0])}") ## because the text is in list
#     x = create_prompt(j[0])
#     text.append(x)
#   except:
#     print(i)
#     continue

In [ ]:
# text[0]

In [38]:
# # use the list text and make new df
# new_train = pd.DataFrame({'text':text})
# # new_train['text'] = text
# new_train

In [39]:
# train['text']

In [40]:
# # train = train[:100]
# text = []
# for i, j in train.iterrows():
#   print(f" row {i},\n\n {create_prompt(j)}")

In [19]:
train

,text
0,<s>[INST] आपको कुछ गणितीय समीकरण दिए गए हैं जो...
1,"<s>[INST] ""वह पांच दिनों से काम करता है"" को सं..."
2,"<s>[INST] दो संख्याओं को दिए गए होने पर, एक गण..."
3,<s>[INST] इस खतरनाक माल के लिए एक चेतावनी लेबल...
4,<s>[INST] पेपर को रीसाइकल करने की प्रक्रिया के...
5,<s>[INST] खुराक बदलने से जुड़े एक खतरा का नाम ...
6,<s>[INST] एक छोड़ी हुई भूलभुलैया में जहां प्रो...
7,<s>[INST] अफ्रीका में सबसे बड़ी झील का नाम बता...


In [18]:
from transformers import TrainingArguments

training_arguments = TrainingArguments(
    output_dir= "/content/drive/MyDrive/results",
    num_train_epochs= 1,
    # max_steps = 1,
    per_device_train_batch_size= 2,
    gradient_accumulation_steps=1,
    save_steps= 1,
    warmup_steps = 0.03,
    logging_steps= 1,
    learning_rate= 2e-4,
    weight_decay= 0.001,
    fp16= False,
    bf16 = False,
    max_grad_norm= 0.3,
    max_steps= -1,
    warmup_ratio= 0.3,
    group_by_length= True,
    lr_scheduler_type= "constant",
    # report_to="wandb",

)


In [34]:
# train_dataset["messages"]

**10000 Rupee note -**
SFT trainer dosesnt takes the input dataframe in the pandas dataframe ,
internally when sft maps the formatting function it cannot map with pandas dataframe, if using the dataset from hf , make sure to use the same dict format

In [35]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset["messages"], # was usin the pandas df here instesd of dict
    peft_config=peft_config,
    max_seq_length=None,
    formatting_func=create_prompt,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=True,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:222: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 512
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1131 > 512). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:282: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


text    [1, 733, 16289, 28793, 28705, 31297, 29794, 29...
Name: 0, dtype: object


In [36]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,1.399800
2,1.391900
3,1.374800
4,1.428000
5,0.942000
6,1.251000


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/

TrainOutput(global_step=6, training_loss=1.2979101936022441, metrics={'train_runtime': 106.6795, 'train_samples_per_second': 0.112, 'train_steps_per_second': 0.056, 'total_flos': 262977777303552.0, 'train_loss': 1.2979101936022441, 'epoch': 1.0})